# imports/setup

In [1]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

# backtester

In [155]:
# edit to mark to mm mid 

In [4]:
class Backtester:
    def __init__(self, trader, listings: Dict[str, Listing], position_limit: Dict[str, int], fair_marks, 
                 market_data: pd.DataFrame, trade_history: pd.DataFrame, file_name: str = None):
        self.trader = trader
        self.listings = listings
        self.market_data = market_data
        self.position_limit = position_limit
        self.fair_marks = fair_marks
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.file_name = file_name

        self.observations = [Observation({}, {}) for _ in range(len(market_data))]

        self.current_position = {product: 0 for product in self.listings.keys()}
        self.pnl_history = []
        self.pnl = {product: 0 for product in self.listings.keys()}
        self.cash = {product: 0 for product in self.listings.keys()}
        self.trades = []
        self.sandbox_logs = []
        
    def run(self):
        traderData = ""
        
        timestamp_group_md = self.market_data.groupby('timestamp')
        timestamp_group_th = self.trade_history.groupby('timestamp')
        
        own_trades = defaultdict(list)
        market_trades = defaultdict(list)
        pnl_product = defaultdict(float)
        
        trade_history_dict = {}
        
        for timestamp, group in timestamp_group_th:
            trades = []
            for _, row in group.iterrows():
                symbol = row['symbol']
                price = row['price']
                quantity = row['quantity']
                buyer = row['buyer'] if pd.notnull(row['buyer']) else ""
                seller = row['seller'] if pd.notnull(row['seller']) else ""

                
                trade = Trade(symbol, int(price), int(quantity), buyer, seller, timestamp)
                
                trades.append(trade)
            trade_history_dict[timestamp] = trades
        
        
        for timestamp, group in timestamp_group_md:
            order_depths = self._construct_order_depths(group)
            order_depths_matching = self._construct_order_depths(group)
            order_depths_pnl = self._construct_order_depths(group)
            state = self._construct_trading_state(traderData, timestamp, self.listings, order_depths, 
                                 dict(own_trades), dict(market_trades), self.current_position, self.observations)
            orders, conversions, traderData = self.trader.run(state)
            products = group['product'].tolist()
            sandboxLog = ""
            trades_at_timestamp = trade_history_dict.get(timestamp, [])

            for product in products:
                new_trades = []
                for order in orders.get(product, []):
                    trades_done, sandboxLog = self._execute_order(timestamp, order, order_depths_matching, self.current_position, self.cash, trade_history_dict, sandboxLog)
                    new_trades.extend(trades_done)
                if len(new_trades) > 0:
                    own_trades[product] = new_trades
            self.sandbox_logs.append({"sandboxLog": sandboxLog, "lambdaLog": "", "timestamp": timestamp})

            trades_at_timestamp = trade_history_dict.get(timestamp, [])
            if trades_at_timestamp:
                for trade in trades_at_timestamp:
                    product = trade.symbol
                    market_trades[product].append(trade)
            else: 
                for product in products:
                    market_trades[product] = []

            
            for product in products:
                self._mark_pnl(self.cash, self.current_position, order_depths_pnl, self.pnl, product)
                self.pnl_history.append(self.pnl[product])
            self._add_trades(own_trades, market_trades)
        return self._log_trades(self.file_name)
    
    
    def _log_trades(self, filename: str = None):
        if filename is None:
            return 

        self.market_data['profit_and_loss'] = self.pnl_history

        output = ""
        output += "Sandbox logs:\n"
        for i in self.sandbox_logs:
            output += json.dumps(i, indent=2) + "\n"

        output += "\n\n\n\nActivities log:\n"
        market_data_csv = self.market_data.to_csv(index=False, sep=";")
        market_data_csv = market_data_csv.replace("\r\n", "\n")
        output += market_data_csv

        output += "\n\n\n\nTrade History:\n"
        output += json.dumps(self.trades, indent=2)

        with open(filename, 'w') as file:
            file.write(output)

            
    def _add_trades(self, own_trades: Dict[str, List[Trade]], market_trades: Dict[str, List[Trade]]):
        products = set(own_trades.keys()) | set(market_trades.keys())
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in own_trades.get(product, [])])
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in market_trades.get(product, [])])

    def _trade_to_dict(self, trade: Trade) -> dict[str, Any]:
        return {
            "timestamp": trade.timestamp,
            "buyer": trade.buyer,
            "seller": trade.seller,
            "symbol": trade.symbol,
            "currency": "SEASHELLS",
            "price": trade.price,
            "quantity": trade.quantity,
        }
        
    def _construct_trading_state(self, traderData, timestamp, listings, order_depths, 
                                 own_trades, market_trades, position, observations):
        state = TradingState(traderData, timestamp, listings, order_depths, 
                             own_trades, market_trades, position, observations)
        return state
    
        
    def _construct_order_depths(self, group):
        order_depths = {}
        for idx, row in group.iterrows():
            product = row['product']
            order_depth = OrderDepth()
            for i in range(1, 4):
                if f'bid_price_{i}' in row and f'bid_volume_{i}' in row:
                    bid_price = row[f'bid_price_{i}']
                    bid_volume = row[f'bid_volume_{i}']
                    if not pd.isna(bid_price) and not pd.isna(bid_volume):
                        order_depth.buy_orders[int(bid_price)] = int(bid_volume)
                if f'ask_price_{i}' in row and f'ask_volume_{i}' in row:
                    ask_price = row[f'ask_price_{i}']
                    ask_volume = row[f'ask_volume_{i}']
                    if not pd.isna(ask_price) and not pd.isna(ask_volume):
                        order_depth.sell_orders[int(ask_price)] = -int(ask_volume)
            order_depths[product] = order_depth
        return order_depths
    
        
        
    def _execute_buy_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]

        for price, volume in list(order_depth.sell_orders.items()):
            if price > order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(trade_volume + position[order.symbol]) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp))
                position[order.symbol] += trade_volume
                self.cash[order.symbol] -= price * trade_volume
                order_depth.sell_orders[price] += trade_volume
                order.quantity -= trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"
            

            if order_depth.sell_orders[price] == 0:
                del order_depth.sell_orders[price]
        
        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price < order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "SUBMISSION", "", timestamp))
                    order.quantity -= trade_volume
                    position[order.symbol] += trade_volume
                    self.cash[order.symbol] -= order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp

        return trades, sandboxLog
        
        
        
    def _execute_sell_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]
        
        for price, volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if price < order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(position[order.symbol] - trade_volume) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "", "SUBMISSION", timestamp))
                position[order.symbol] -= trade_volume
                self.cash[order.symbol] += price * abs(trade_volume)
                order_depth.buy_orders[price] -= abs(trade_volume)
                order.quantity += trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"

            if order_depth.buy_orders[price] == 0:
                del order_depth.buy_orders[price]

        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price > order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "", "SUBMISSION", timestamp))
                    order.quantity += trade_volume
                    position[order.symbol] -= trade_volume
                    self.cash[order.symbol] += order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp
                
        return trades, sandboxLog
        
        
        
    def _execute_order(self, timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog):
        if order.quantity == 0:
            return []
        
        order_depth = order_depths[order.symbol]
        if order.quantity > 0:
            return self._execute_buy_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
        else:
            return self._execute_sell_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
    
    def _mark_pnl(self, cash, position, order_depths, pnl, product):
        order_depth = order_depths[product]
        
        best_ask = min(order_depth.sell_orders.keys())
        best_bid = max(order_depth.buy_orders.keys())
        mid = (best_ask + best_bid)/2
        fair = mid
        if product in self.fair_marks:
            get_fair = self.fair_marks[product]
            fair = get_fair(order_depth)
        
        pnl[product] = cash[product] + fair * position[product]
        


# trader

In [5]:
from datamodel import OrderDepth, UserId, TradingState, Order
from typing import List
import string
import jsonpickle
import numpy as np
import math


class Product:
    AMETHYSTS = "RAINFOREST_RESIN"
    STARFRUIT = "KELP"


PARAMS = {
    Product.AMETHYSTS: {
        "fair_value": 10000,
        "take_width": 1,
        "clear_width": 0,
        # for making
        "disregard_edge": 1,  # disregards orders for joining or pennying within this value from fair
        "join_edge": 2,  # joins orders within this edge
        "default_edge": 4,
        "soft_position_limit": 10,
    },
    Product.STARFRUIT: {
        "take_width": 1,
        "clear_width": -0.25,
        "prevent_adverse": True,
        "adverse_volume": 15,
        "reversion_beta": -0.229,
        "disregard_edge": 1,
        "join_edge": 0,
        "default_edge": 1,
    },
}


class Trader:
    def __init__(self, params=None):
        if params is None:
            params = PARAMS
        self.params = params

        self.LIMIT = {Product.AMETHYSTS: 20, Product.STARFRUIT: 20}

    def take_best_orders(
        self,
        product: str,
        fair_value: int,
        take_width: float,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
        prevent_adverse: bool = False,
        adverse_volume: int = 0,
    ) -> (int, int):
        position_limit = self.LIMIT[product]

        if len(order_depth.sell_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_ask_amount = -1 * order_depth.sell_orders[best_ask]

            if not prevent_adverse or abs(best_ask_amount) <= adverse_volume:
                if best_ask <= fair_value - take_width:
                    quantity = min(
                        best_ask_amount, position_limit - position
                    )  # max amt to buy
                    if quantity > 0:
                        orders.append(Order(product, best_ask, quantity))
                        buy_order_volume += quantity
                        order_depth.sell_orders[best_ask] += quantity
                        if order_depth.sell_orders[best_ask] == 0:
                            del order_depth.sell_orders[best_ask]

        if len(order_depth.buy_orders) != 0:
            best_bid = max(order_depth.buy_orders.keys())
            best_bid_amount = order_depth.buy_orders[best_bid]

            if not prevent_adverse or abs(best_bid_amount) <= adverse_volume:
                if best_bid >= fair_value + take_width:
                    quantity = min(
                        best_bid_amount, position_limit + position
                    )  # should be the max we can sell
                    if quantity > 0:
                        orders.append(Order(product, best_bid, -1 * quantity))
                        sell_order_volume += quantity
                        order_depth.buy_orders[best_bid] -= quantity
                        if order_depth.buy_orders[best_bid] == 0:
                            del order_depth.buy_orders[best_bid]

        return buy_order_volume, sell_order_volume

    def market_make(
        self,
        product: str,
        orders: List[Order],
        bid: int,
        ask: int,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> (int, int):
        buy_quantity = self.LIMIT[product] - (position + buy_order_volume)
        if buy_quantity > 0:
            orders.append(Order(product, round(bid), buy_quantity))  # Buy order

        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)
        if sell_quantity > 0:
            orders.append(Order(product, round(ask), -sell_quantity))  # Sell order
        return buy_order_volume, sell_order_volume

    def clear_position_order(
        self,
        product: str,
        fair_value: float,
        width: int,
        orders: List[Order],
        order_depth: OrderDepth,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> List[Order]:
        position_after_take = position + buy_order_volume - sell_order_volume
        fair_for_bid = round(fair_value - width)
        fair_for_ask = round(fair_value + width)

        buy_quantity = self.LIMIT[product] - (position + buy_order_volume)
        sell_quantity = self.LIMIT[product] + (position - sell_order_volume)

        if position_after_take > 0:
            # Aggregate volume from all buy orders with price greater than fair_for_ask
            clear_quantity = sum(
                volume
                for price, volume in order_depth.buy_orders.items()
                if price >= fair_for_ask
            )
            clear_quantity = min(clear_quantity, position_after_take)
            sent_quantity = min(sell_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_ask, -abs(sent_quantity)))
                sell_order_volume += abs(sent_quantity)

        if position_after_take < 0:
            # Aggregate volume from all sell orders with price lower than fair_for_bid
            clear_quantity = sum(
                abs(volume)
                for price, volume in order_depth.sell_orders.items()
                if price <= fair_for_bid
            )
            clear_quantity = min(clear_quantity, abs(position_after_take))
            sent_quantity = min(buy_quantity, clear_quantity)
            if sent_quantity > 0:
                orders.append(Order(product, fair_for_bid, abs(sent_quantity)))
                buy_order_volume += abs(sent_quantity)

        return buy_order_volume, sell_order_volume

    def starfruit_fair_value(self, order_depth: OrderDepth, traderObject) -> float:
        if len(order_depth.sell_orders) != 0 and len(order_depth.buy_orders) != 0:
            best_ask = min(order_depth.sell_orders.keys())
            best_bid = max(order_depth.buy_orders.keys())
            filtered_ask = [
                price
                for price in order_depth.sell_orders.keys()
                if abs(order_depth.sell_orders[price])
                >= self.params[Product.STARFRUIT]["adverse_volume"]
            ]
            filtered_bid = [
                price
                for price in order_depth.buy_orders.keys()
                if abs(order_depth.buy_orders[price])
                >= self.params[Product.STARFRUIT]["adverse_volume"]
            ]
            mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else None
            mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else None
            if mm_ask == None or mm_bid == None:
                if traderObject.get("starfruit_last_price", None) == None:
                    mmmid_price = (best_ask + best_bid) / 2
                else:
                    mmmid_price = traderObject["starfruit_last_price"]
            else:
                mmmid_price = (mm_ask + mm_bid) / 2

            if traderObject.get("starfruit_last_price", None) != None:
                last_price = traderObject["starfruit_last_price"]
                last_returns = (mmmid_price - last_price) / last_price
                pred_returns = (
                    last_returns * self.params[Product.STARFRUIT]["reversion_beta"]
                )
                fair = mmmid_price + (mmmid_price * pred_returns)
            else:
                fair = mmmid_price
            traderObject["starfruit_last_price"] = mmmid_price
            return fair
        return None

    def take_orders(
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        take_width: float,
        position: int,
        prevent_adverse: bool = False,
        adverse_volume: int = 0,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume = 0
        sell_order_volume = 0

        buy_order_volume, sell_order_volume = self.take_best_orders(
            product,
            fair_value,
            take_width,
            orders,
            order_depth,
            position,
            buy_order_volume,
            sell_order_volume,
            prevent_adverse,
            adverse_volume,
        )
        return orders, buy_order_volume, sell_order_volume

    def clear_orders(
        self,
        product: str,
        order_depth: OrderDepth,
        fair_value: float,
        clear_width: int,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
    ) -> (List[Order], int, int):
        orders: List[Order] = []
        buy_order_volume, sell_order_volume = self.clear_position_order(
            product,
            fair_value,
            clear_width,
            orders,
            order_depth,
            position,
            buy_order_volume,
            sell_order_volume,
        )
        return orders, buy_order_volume, sell_order_volume

    def make_orders(
        self,
        product,
        order_depth: OrderDepth,
        fair_value: float,
        position: int,
        buy_order_volume: int,
        sell_order_volume: int,
        disregard_edge: float,  # disregard trades within this edge for pennying or joining
        join_edge: float,  # join trades within this edge
        default_edge: float,  # default edge to request if there are no levels to penny or join
        manage_position: bool = False,
        soft_position_limit: int = 0,
        # will penny all other levels with higher edge
    ):
        orders: List[Order] = []
        asks_above_fair = [
            price
            for price in order_depth.sell_orders.keys()
            if price > fair_value + disregard_edge
        ]
        bids_below_fair = [
            price
            for price in order_depth.buy_orders.keys()
            if price < fair_value - disregard_edge
        ]

        best_ask_above_fair = min(asks_above_fair) if len(asks_above_fair) > 0 else None
        best_bid_below_fair = max(bids_below_fair) if len(bids_below_fair) > 0 else None

        ask = round(fair_value + default_edge)
        if best_ask_above_fair != None:
            if abs(best_ask_above_fair - fair_value) <= join_edge:
                ask = best_ask_above_fair  # join
            else:
                ask = best_ask_above_fair - 1  # penny

        bid = round(fair_value - default_edge)
        if best_bid_below_fair != None:
            if abs(fair_value - best_bid_below_fair) <= join_edge:
                bid = best_bid_below_fair
            else:
                bid = best_bid_below_fair + 1

        if manage_position:
            if position > soft_position_limit:
                ask -= 1
            elif position < -1 * soft_position_limit:
                bid += 1

        buy_order_volume, sell_order_volume = self.market_make(
            product,
            orders,
            bid,
            ask,
            position,
            buy_order_volume,
            sell_order_volume,
        )

        return orders, buy_order_volume, sell_order_volume

    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData != None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)

        result = {}

        if Product.AMETHYSTS in self.params and Product.AMETHYSTS in state.order_depths:
            amethyst_position = (
                state.position[Product.AMETHYSTS]
                if Product.AMETHYSTS in state.position
                else 0
            )
            amethyst_take_orders, buy_order_volume, sell_order_volume = (
                self.take_orders(
                    Product.AMETHYSTS,
                    state.order_depths[Product.AMETHYSTS],
                    self.params[Product.AMETHYSTS]["fair_value"],
                    self.params[Product.AMETHYSTS]["take_width"],
                    amethyst_position,
                )
            )
            amethyst_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.AMETHYSTS,
                    state.order_depths[Product.AMETHYSTS],
                    self.params[Product.AMETHYSTS]["fair_value"],
                    self.params[Product.AMETHYSTS]["clear_width"],
                    amethyst_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            amethyst_make_orders, _, _ = self.make_orders(
                Product.AMETHYSTS,
                state.order_depths[Product.AMETHYSTS],
                self.params[Product.AMETHYSTS]["fair_value"],
                amethyst_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.AMETHYSTS]["disregard_edge"],
                self.params[Product.AMETHYSTS]["join_edge"],
                self.params[Product.AMETHYSTS]["default_edge"],
                True,
                self.params[Product.AMETHYSTS]["soft_position_limit"],
            )
            result[Product.AMETHYSTS] = (
                amethyst_take_orders + amethyst_clear_orders + amethyst_make_orders
            )

        if Product.STARFRUIT in self.params and Product.STARFRUIT in state.order_depths:
            starfruit_position = (
                state.position[Product.STARFRUIT]
                if Product.STARFRUIT in state.position
                else 0
            )
            starfruit_fair_value = self.starfruit_fair_value(
                state.order_depths[Product.STARFRUIT], traderObject
            )
            starfruit_take_orders, buy_order_volume, sell_order_volume = (
                self.take_orders(
                    Product.STARFRUIT,
                    state.order_depths[Product.STARFRUIT],
                    starfruit_fair_value,
                    self.params[Product.STARFRUIT]["take_width"],
                    starfruit_position,
                    self.params[Product.STARFRUIT]["prevent_adverse"],
                    self.params[Product.STARFRUIT]["adverse_volume"],
                )
            )
            starfruit_clear_orders, buy_order_volume, sell_order_volume = (
                self.clear_orders(
                    Product.STARFRUIT,
                    state.order_depths[Product.STARFRUIT],
                    starfruit_fair_value,
                    self.params[Product.STARFRUIT]["clear_width"],
                    starfruit_position,
                    buy_order_volume,
                    sell_order_volume,
                )
            )
            starfruit_make_orders, _, _ = self.make_orders(
                Product.STARFRUIT,
                state.order_depths[Product.STARFRUIT],
                starfruit_fair_value,
                starfruit_position,
                buy_order_volume,
                sell_order_volume,
                self.params[Product.STARFRUIT]["disregard_edge"],
                self.params[Product.STARFRUIT]["join_edge"],
                self.params[Product.STARFRUIT]["default_edge"],
            )
            result[Product.STARFRUIT] = (
                starfruit_take_orders + starfruit_clear_orders + starfruit_make_orders
            )

        conversions = 1
        traderData = jsonpickle.encode(traderObject)

        return result, conversions, traderData


# backtest run

In [6]:
from datamodel import Listing, Observation
import pandas as pd
import json
import io

In [7]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

In [243]:
# from round_1_parameterized_v3 import Trader
# from round_1_cleaned_v2 import Trader

### setup

In [8]:
def calculate_starfruit_fair(order_depth):
    # assumes order_depth has orders in it 
    best_ask = min(order_depth.sell_orders.keys())
    best_bid = max(order_depth.buy_orders.keys())
    filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
    filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
    mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
    mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

    mmmid_price = (mm_ask + mm_bid) / 2
    return mmmid_price
    
def calculate_amethysts_fair(order_depth):
    return 10000

In [10]:
listings = {
    'AMETHYSTS': Listing(symbol='AMETHYSTS', product='AMETHYSTS', denomination='SEASHELLS'),
    'STARFRUIT': Listing(symbol='STARFRUIT', product='STARFRUIT', denomination='SEASHELLS')
}

position_limit = {
    'AMETHYSTS': 20,
    'STARFRUIT': 20
}

fair_calculations = {
    "AMETHYSTS": calculate_amethysts_fair,
    "STARFRUIT": calculate_starfruit_fair
}


In [11]:
position_limit

{'AMETHYSTS': 20, 'STARFRUIT': 20}

In [ ]:
market_data, trade_history = _process_data_('round_1_clean_data.log')
trader = Trader()
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "backtests/yuh")
backtester.run()
print(backtester.pnl)

In [249]:
# with fair prediction
day = 0
market_data = pd.read_csv(f"./round-1-island-data-bottle/prices_round_1_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round-1-island-data-bottle/trades_round_1_day_{day}_nn.csv", sep=";", header=0)

trader = Trader()
backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "backtests/volume.log")
backtester.run()
print(backtester.pnl)


{'AMETHYSTS': 13940, 'STARFRUIT': 14564.0}


`{'AMETHYSTS': 14554, 'STARFRUIT': 14144.5}`

In [ ]:
day = 0
market_data = pd.read_csv(f"./round_0_data/prices_round_0_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round_0_data/trades_round_0_day_{day}_nn.csv", sep=";", header=0)

trader = Trader()
backtester = Backtester(trader, listings, position_limit, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)

# backtest gridsearch

In [36]:
import itertools

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [34]:
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        print(params)
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [9]:
listings = {
    'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
    'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS')
}

position_limit = {
    'RAINFOREST_RESIN': 20,
    'KELP': 20
}

fair_calculations = {
    "RAINFOREST_RESIN": calculate_amethysts_fair,
    "KELP": calculate_starfruit_fair
}


In [10]:
day = 0
market_data = pd.read_csv(f"./round_0_data/prices_round_0_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"./round_0_data/trades_round_0_day_{day}_nn.csv", sep=";", header=0)

### run

In [37]:
backtest_dir = "backtest_test_clear_width"

param_grid = {
    Product.AMETHYSTS: {
        "fair_value": [10000],
        "take_width": [0.5, 1, 1.5],  
        "clear_width": [0.25, 0.5, 0.75],  
        "volume_limit": [0, 5, 10],  
        "disregard_edge": [0.5, 1, 1.5],  
        "join_edge": [1, 2, 3],  
        "default_edge": [3, 4, 5], 
        "soft_position_limit": [5, 10, 15]  
    },
    Product.STARFRUIT: {
        "take_width": [0.5, 1, 1.5],  
        "clear_width": [-0.5, -0.25, 0, 0.25], 
        "prevent_adverse": [True, False],  
        "adverse_volume": [10, 15, 20],  
        "reversion_beta": [-0.3, -0.229, -0.15],  
        "disregard_edge": [0.5, 1, 1.5],  
        "join_edge": [2, 3, 4],  
        "default_edge": [4, 5, 6]  
    }
}



trader = Trader()

amethyst_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "RAINFOREST_RESIN")
print("AMETHYSTS results:")
for params, pnl in amethyst_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

starfruit_results = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "KELP")
print("STARFRUIT results:")
for params, pnl in starfruit_results: 
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

Running backtests for RAINFOREST_RESIN:   0%|          | 0/2187 [00:00<?, ?backtest/s]

{'fair_value': 10000, 'take_width': 0.5, 'clear_width': 0.25, 'volume_limit': 0, 'disregard_edge': 0.5, 'join_edge': 1, 'default_edge': 3, 'soft_position_limit': 5}


Running backtests for RAINFOREST_RESIN:   0%|          | 1/2187 [00:04<2:48:11,  4.62s/backtest]

{'fair_value': 10000, 'take_width': 0.5, 'clear_width': 0.25, 'volume_limit': 0, 'disregard_edge': 0.5, 'join_edge': 1, 'default_edge': 3, 'soft_position_limit': 10}


Running backtests for RAINFOREST_RESIN:   0%|          | 1/2187 [00:06<3:47:39,  6.25s/backtest]


KeyboardInterrupt: 

## analyze

In [ ]:
import os
backtest_dir = "backtest_test_clear_width"
def analyze_log_files(backtest_dir):
    log_files = [f for f in os.listdir(backtest_dir) if f.endswith('.log')]
    
    results = []
    for log_file in log_files:
        file_path = os.path.join(backtest_dir, log_file)
        
        # Extract symbol and parameters from the file name
        file_name = os.path.splitext(log_file)[0]
        print(file_name)
        symbol, params_str = file_name.split('-', 1)
        params = dict(param.split('=') for param in params_str.split('-'))
        
        # Read the contents of the log file
        with open(file_path, 'r') as file:
            log_content = file.read()
        
        # Store the symbol, parameters, and log content in the results
        results.append({
            'symbol': symbol,
            'params': params,
            'log_content': log_content
        })
    
    return results

# Analyze the log files
log_analysis_results = analyze_log_files(backtest_dir)

# Print the results
for result in log_analysis_results:
    print(f"Symbol: {result['symbol']}")
    print(f"Parameters: {result['params']}")
#     print(f"Log Content:\n{result['log_content']}\n")

In [ ]:
sorted_starfruit_results = sorted(starfruit_results, key=lambda x: x[1], reverse=True)


In [239]:
sorted_starfruit_results[0:100]

[({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 1},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 2},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 3},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'default_edge': 4},
  14808.0),
 ({'take_width': 1,
   'clear_width': -0.25,
   'prevent_adverse': True,
   'adverse_volume': 15,
   'reversion_beta': -0.229,
   'disregard_edge': 1,
   'join_edge': 0,
   'defaul

In [38]:
import optuna
backtest_dir = "backtest"
import os
from tqdm import tqdm

def run_backtests(trader, listings, position_limit, fair_calcs, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    results = []
    trader.params = param_grid[symbol]
    backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
    backtester.run()

    param_str = "-".join([f"{key}={value}" for key, value in param_grid[symbol].items()])
    log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
    backtester._log_trades(log_filename)

    return param_grid[symbol], backtester.pnl[symbol]

def objective(trial):
    param_grid = {
        Product.AMETHYSTS: {
            "fair_value": 10000,
            'take_width': trial.suggest_float('amethyst_take_width', 0.5, 10.0, step = 0.5),
            'clear_width': trial.suggest_float('amethyst_clear_width', -0.5, 1, step = 0.1),
            'volume_limit': trial.suggest_int('amethyst_volume_limit', 0, 20, step = 5),
            'disregard_edge': trial.suggest_float('amethyst_disregard_edge', 0.5, 5.0, step = 0.5),
            'join_edge': trial.suggest_float('amethyst_join_edge', 1.0, 5.0, step = 1),
            'default_edge': trial.suggest_float('amethyst_default_edge', 3.0, 10.0, step = 1),
            'soft_position_limit': trial.suggest_int('amethyst_soft_position_limit', 5, 20, step = 5)
        },
        Product.STARFRUIT: {
            'take_width': trial.suggest_float('starfruit_take_width', 0.5, 10.0, step = 0.5),
            'clear_width': trial.suggest_float('starfruit_clear_width', -0.5, 1, step = 0.1),
            'prevent_adverse': trial.suggest_categorical('starfruit_prevent_adverse', [True, False]),
            'adverse_volume': trial.suggest_int('starfruit_adverse_volume', 10, 20),
            'reversion_beta': trial.suggest_float('starfruit_reversion_beta', -0.5, 0.5, step = 0.1),
            'disregard_edge': trial.suggest_float('starfruit_disregard_edge', 0.5, 5.0, step = 0.5),
            'join_edge': trial.suggest_float('starfruit_join_edge', 1.0, 5.0, step = 1),
            'default_edge': trial.suggest_float('starfruit_default_edge', 3.0, 10.0, step = 1)
        }
    }
    
    trader = Trader()
    params, pnl_1 = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "RAINFOREST_RESIN")
    print("AMETHYSTS results:")

    print(params)
    print(f"pnl: {pnl}")
    print("="*80)

    params, pnl_2 = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_history, backtest_dir, param_grid, "KELP")
    print("STARFRUIT results:")
    print(params)
    print(f"pnl: {pnl}")
    print("="*80)
    # except Exception as e:
    #     print(f"Trial failed: {e}")
    #     return float('-inf')
    return pnl_1 + pnl_2

# Create and run the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

# Print results
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
print(study.best_params)
df = study.trials_dataframe()
df.to_csv(f"./backtest/backtest_0.csv")

# Visualize results
import optuna.visualization as vis
vis.plot_optimization_history(study)
vis.plot_param_importances(study)

[I 2025-03-23 10:24:35,799] A new study created in memory with name: no-name-cffca6fb-c238-4255-a249-81aa8698fc02


{'RAINFOREST_RESIN': 0, 'KELP': 0.0}
AMETHYSTS results:
{'fair_value': 10000, 'take_width': 9.5, 'clear_width': 0.6000000000000001, 'volume_limit': 10, 'disregard_edge': 5.0, 'join_edge': 5.0, 'default_edge': 3.0, 'soft_position_limit': 5}
pnl: 0


[W 2025-03-23 10:24:45,029] Trial 0 failed with parameters: {'amethyst_take_width': 9.5, 'amethyst_clear_width': 0.6000000000000001, 'amethyst_volume_limit': 10, 'amethyst_disregard_edge': 5.0, 'amethyst_join_edge': 5.0, 'amethyst_default_edge': 3.0, 'amethyst_soft_position_limit': 5, 'starfruit_take_width': 2.5, 'starfruit_clear_width': -0.4, 'starfruit_prevent_adverse': False, 'starfruit_adverse_volume': 11, 'starfruit_reversion_beta': 0.5, 'starfruit_disregard_edge': 4.5, 'starfruit_join_edge': 3.0, 'starfruit_default_edge': 3.0} because of the following error: The value None could not be cast to float..
[W 2025-03-23 10:24:45,029] Trial 0 failed with value None.


{'RAINFOREST_RESIN': 0, 'KELP': 0.0}
STARFRUIT results:
{'take_width': 2.5, 'clear_width': -0.4, 'prevent_adverse': False, 'adverse_volume': 11, 'reversion_beta': 0.5, 'disregard_edge': 4.5, 'join_edge': 3.0, 'default_edge': 3.0}
pnl: 0.0
{'RAINFOREST_RESIN': 0, 'KELP': 0.0}
AMETHYSTS results:
{'fair_value': 10000, 'take_width': 4.5, 'clear_width': -0.09999999999999998, 'volume_limit': 10, 'disregard_edge': 1.5, 'join_edge': 5.0, 'default_edge': 9.0, 'soft_position_limit': 10}
pnl: 0


[W 2025-03-23 10:24:53,654] Trial 1 failed with parameters: {'amethyst_take_width': 4.5, 'amethyst_clear_width': -0.09999999999999998, 'amethyst_volume_limit': 10, 'amethyst_disregard_edge': 1.5, 'amethyst_join_edge': 5.0, 'amethyst_default_edge': 9.0, 'amethyst_soft_position_limit': 10, 'starfruit_take_width': 9.0, 'starfruit_clear_width': 0.9000000000000001, 'starfruit_prevent_adverse': False, 'starfruit_adverse_volume': 13, 'starfruit_reversion_beta': 0.10000000000000009, 'starfruit_disregard_edge': 1.0, 'starfruit_join_edge': 5.0, 'starfruit_default_edge': 3.0} because of the following error: The value None could not be cast to float..
[W 2025-03-23 10:24:53,654] Trial 1 failed with value None.


...

{'RAINFOREST_RESIN': 0, 'KELP': 0.0}
STARFRUIT results:
{'take_width': 7.5, 'clear_width': 0.30000000000000004, 'prevent_adverse': False, 'adverse_volume': 20, 'reversion_beta': -0.09999999999999998, 'disregard_edge': 1.5, 'join_edge': 3.0, 'default_edge': 5.0}
pnl: 0.0
{'RAINFOREST_RESIN': 0, 'KELP': 0.0}
AMETHYSTS results:
{'fair_value': 10000, 'take_width': 7.5, 'clear_width': -0.19999999999999996, 'volume_limit': 20, 'disregard_edge': 0.5, 'join_edge': 4.0, 'default_edge': 4.0, 'soft_position_limit': 15}
pnl: 0


[W 2025-03-23 10:25:12,069] Trial 3 failed with parameters: {'amethyst_take_width': 7.5, 'amethyst_clear_width': -0.19999999999999996, 'amethyst_volume_limit': 20, 'amethyst_disregard_edge': 0.5, 'amethyst_join_edge': 4.0, 'amethyst_default_edge': 4.0, 'amethyst_soft_position_limit': 15, 'starfruit_take_width': 1.0, 'starfruit_clear_width': -0.5, 'starfruit_prevent_adverse': False, 'starfruit_adverse_volume': 20, 'starfruit_reversion_beta': -0.09999999999999998, 'starfruit_disregard_edge': 3.0, 'starfruit_join_edge': 2.0, 'starfruit_default_edge': 10.0} because of the following error: The value None could not be cast to float..
[W 2025-03-23 10:25:12,069] Trial 3 failed with value None.


{'RAINFOREST_RESIN': 0, 'KELP': 0.0}
STARFRUIT results:
{'take_width': 1.0, 'clear_width': -0.5, 'prevent_adverse': False, 'adverse_volume': 20, 'reversion_beta': -0.09999999999999998, 'disregard_edge': 3.0, 'join_edge': 2.0, 'default_edge': 10.0}
pnl: 0.0
{'RAINFOREST_RESIN': 0, 'KELP': 0.0}
AMETHYSTS results:
{'fair_value': 10000, 'take_width': 9.0, 'clear_width': 0.20000000000000007, 'volume_limit': 10, 'disregard_edge': 4.5, 'join_edge': 4.0, 'default_edge': 5.0, 'soft_position_limit': 15}
pnl: 0


[W 2025-03-23 10:25:17,768] Trial 4 failed with parameters: {'amethyst_take_width': 9.0, 'amethyst_clear_width': 0.20000000000000007, 'amethyst_volume_limit': 10, 'amethyst_disregard_edge': 4.5, 'amethyst_join_edge': 4.0, 'amethyst_default_edge': 5.0, 'amethyst_soft_position_limit': 15, 'starfruit_take_width': 3.0, 'starfruit_clear_width': 0.10000000000000009, 'starfruit_prevent_adverse': False, 'starfruit_adverse_volume': 19, 'starfruit_reversion_beta': 0.10000000000000009, 'starfruit_disregard_edge': 2.5, 'starfruit_join_edge': 2.0, 'starfruit_default_edge': 7.0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\IMC_prosperity\.venv\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\luong\AppData\Local\Temp\ipykernel_25088\2791979166.py", line 56, in objective
    params, pnl = run_backtests(trader, listings, position_limit, fair_calculations, market_data, trade_hist

KeyboardInterrupt: 